# Functional Programming Techniques in Python Part 1

Author:[mkocher](https://github.com/mpkocher)

This is introduction to Functional Programming Techniques (FPT) using Python. 

Topics Covered

- Function basics
- Closures
- Partial Application (from functools)
- Reduce (from functools)
- Function Composition
- Map and Filter from stdlib
- Creating Data Pipelines using FPT
- Conclusion and Summary

In [1]:
import datetime
import types
import string
import random
from collections.abc import Iterable, Callable
# Requires Python >= 3.7
from dataclasses import dataclass, fields

import functools
import itertools
import operator as op

In [2]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-15 03:19:07.472785


#### Demo Data

Define some utils for generating test data. 

In [3]:
XS = range(10)

@dataclass(frozen=True)
class User(object):
    user_id:int
    age:int    
    first_name:str
    favorite_color:str  

        
def to_users():
    colors = ['red', "blue", "black", "orange", "yellow"]
    def to_random(xs):
        return random.choice(xs)
    to_color = functools.partial(to_random, colors)
    
    names = [("Ralph", 13), ("Joe", 43),("Steve", 66), ("Rebecca", 41), 
             ("Sam", 4), ("Richard", 32), ("Paul", 87), 
             ("Stephen", 55), ("Sean", 2)]

    for i, (name, age) in enumerate(names):
        yield User(i + 1, age, name, to_color())

# 1. Python Function Basics

There are several different models for defining functions in Python.

The first is `lambda` functions. This model should primarly be used within function calls. 

A good discussion [in the Python docs](https://docs.python.org/3.7/howto/functional.html#small-functions-and-the-lambda-expression) provides some general tips on when use the `lambda` style. 

In [4]:
f0 = lambda n: n * n

f0(3) == 9

print(f0(3))
print(f0)
print(f0.__name__)

9
<function <lambda> at 0x10749cd08>
<lambda>


The most common model to write functions in Python show below. 

In [5]:
def f1(n):
    """A simple function to square a number"""
    return n * n

# Or using type annotations in Python 3
def f2(n:int) -> int:
    return n * n

assert f1(3) == f2(3)
print(f1(3))
print((f1.__name__, f1.__doc__))
print((f1, f2))


9
('f1', 'A simple function to square a number')
(<function f1 at 0x10749ce18>, <function f2 at 0x10749cea0>)


Alternatively, classes can be used to create `Callables`. This can be useful when you need to initialize a class with state.

In [6]:
# Or using a class style. 
class F3(Callable):
    def __call__(self, n):
        return n * n
    
f3 = F3()
print(("Callable class {} {}".format(f3, f3(3))))

assert f3(3) == f1(3)

# Perhaps a better example. 
class F4(Callable):
    def __init__(self, n):
        self.n = n
    
    def __repr__(self):
        _d = dict(k=self.__class__.__name__, n=self.n)
        return "<{k} n:{n}>".format(**_d)
    
    def __call__(self, n):
        return self.n * n
    
    def compute(self, n):
        return self.n * n

f4 = F4(2)
print(f4)

assert f4(3) == 6

# You can also define a bound function
f5 = f4.compute
print(f5)

assert f5(3) == 6

Callable class <__main__.F3 object at 0x1074eaf60> 9
<F4 n:2>
<bound method F4.compute of <F4 n:2>>


This adds quite a bit of boilerplate. There's a cleaner technique leveraging the first class nature of functions in Python. 

# 2. Closures 

In Python we can define functions that return functions. 

For example:

In [7]:
def multipler(x):
    def func(y):
        return x * y
    return func

f6 = multipler(2)
print("Func {} out={}".format(f6, f6(3)))

Func <function multipler.<locals>.func at 0x1074ee400> out=6


This is an extremely value pattern to leverage, specifically if you have "pure" (i.e. side-effect free) functions. This can be thought of as a "factory" that returns a unit computation. It's also information hidding and echos the features of the class approach shown above. 

Here's a few examples to help demonstrate the pattern. Let's filter out all the values of list using the list comprehension style.

In [8]:
def lte_3(n):
    return n <= 3

[x for x in XS if lte_3(x)]

[0, 1, 2, 3]

Let's generalize our filter func so that we can leverage this util in other areas of our code base.

In [9]:
def to_filter(min_value):
    def func(x):
        return x <= min_value
    return func

filter_3 = to_filter(3)
[x for x in XS if filter_3(x)]

[0, 1, 2, 3]

This is very powerful technique. The examples shown here are minimal to demonstrate the core idea. 


## Digression

Let's look at the `lru_cache` from the Python standard lib to look at some real world examples using closures as a core design pattern.

https://github.com/python/cpython/blob/3.7/Lib/functools.py#L445

There's a few interesting points to note.

1. The use of bound functions to avoid the extra dictionary lookup (the `lrc_cache` is often used in a tight loop).

https://github.com/python/cpython/blob/3.7/Lib/functools.py#L494

```python
cache_get = cache.get
```

2. The use of `nonlocal` to enable mutating the number of hits and misses within a closure. 

https://github.com/python/cpython/blob/3.7/Lib/functools.py#L513


More details on the use of `nonlocal` in the [official docs](https://docs.python.org/3/reference/simple_stmts.html#nonlocal).

https://github.com/python/cpython/blob/3.7/Lib/functools.py#L597


```python
nonlocal hits, misses
```

In Python 2.7.x, you can hack around this example by using a mutable container. 

For example:

In [10]:
def two_seven(n):
    # DONT do this in Python 3. Use nonlocal.
    total = [0]
    
    def f(m):
        total[0] += 1
        return n + m
    
    def get_total():
        return total[0]
    
    # see comments below about use of this pattern
    f.get_total = get_total
    return f

def two_seven_example():
    f27 = two_seven(2)
    def fx():
        return f27(1)
    for _ in range(5):
        for name, func in (("calling func", fx), ("total times called", f27.get_total)):
            print("{}={}".format(name, func()))

two_seven_example()

calling func=3
total times called=1
calling func=3
total times called=2
calling func=3
total times called=3
calling func=3
total times called=4
calling func=3
total times called=5


3. Another design choice used in `lru_cache` is attach functions to the wrapper function used in the closure. 

https://github.com/python/cpython/blob/3.7/Lib/functools.py#L597

```python
wrapper.cache_info = cache_info
wrapper.cache_clear = cache_clear
return wrapper
```

I suspect the choice of using functions instead of class here is for performance reasons. 

In general, this pattern should be used judiciously. Often it's probably better to migrate to use classes for these cases instead of functions and closures. 

The often quoted `objects are truly a poor man's closures .... Closures are a poor man's object.` is an interesting read.

http://wiki.c2.com/?ClosuresAndObjectsAreEquivalent


## Other Examples of API Design Leveraging Closures

[More involved examples from production code are shown here](https://github.com/mpkocher/pbcommand/blob/master/pbcommand/cli/quick.py#L249).

[And here's a snippet using argparse's Action](https://github.com/mpkocher/pbcommand/blob/master/pbcommand/common_options.py#L109).

In [11]:
import argparse

def _to_print_message_action(msg):

    class PrintMessageAction(argparse.Action):

        """Print message and exit"""

        def __call__(self, parser, namespace, values, option_string=None):
            sys.stdout.write(msg + "\n")
            sys.exit(0)

    return PrintMessageAction

# 3. Partial Application and Functools.partial

Using closures is a useful pattern, however, we often want to write our functions without this extra function returning layer. This can be addressed using a techinque known as *Partial Application*. This is accessible in [Python's standard library](https://docs.python.org/3/library/functools.html#functools.partial) using `functools.partial`. 

Let's revisiting the `to_filter` example.

In [12]:
def to_filter(min_value):
    def func(x):
        return x <= min_value
    return func

# Let's rewrite this in a more "natural" form.

def filter_min_value(min_value, x):
    return x <= min_value

for x, y in [(2, 4),(4, 3)]:
    print("min={} x={} {}".format(x, y, filter_min_value(x, y)))

min=2 x=4 False
min=4 x=3 True


Now let's use `functools.partial` to bind the first val in the function and return a new function. 

In [13]:
filter_lte_3 = functools.partial(filter_min_value, 3)
print(filter_lte_3)

[x for x in XS if filter_lte_3(x)]

functools.partial(<function filter_min_value at 0x1074eeb70>, 3)


[0, 1, 2, 3]

For some cases the partial application approach can reduce the boilerplate. 

However, there is one important note to be aware of. When using `functools.partial`, the returned func will not have `__doc__` defined. Sometimes it might be useful to explicitly set `__doc__` on the func to have closer parity to "standard" functions. 

In [14]:
print(filter_min_value.__name__)

filter_min_value


In [15]:
print(filter_lte_3.__name__)

AttributeError: 'functools.partial' object has no attribute '__name__'

In [16]:
filter_lte_3.__name__ = "Filter lte 3"
print(filter_lte_3.__name__)

Filter lte 3


One more important util from `functools` is `reduce`. 

# 4. Reduce using functools.reduce

This is used to take a `list[T]` and reduce (or 'fold') to a single result (which is not necessarily type `T`).

General Form:

`f(acc, value, init_value=None) -> value`

The init value allows you to set the initial value to reduce on. Note this fundamentally differs from `map` and `filter` which return an iterable. 

See the [official docs](https://docs.python.org/3/library/functools.html#functools.reduce) for more details.

Here's a few examples:

In [17]:
def adder(a, b):
    return a + b

functools.reduce(adder, XS)

45

In [18]:
# Or using `operator` from the standard lib https://docs.python.org/3/library/operator.html
functools.reduce(op.add, XS)

45

In [19]:
init_value = 11
functools.reduce(op.add, range(10), init_value)

56

In [20]:
functools.reduce(op.mul, range(1, 10))

362880

Note this can be used with custom classes as well. 

Here's a bit of a contrived example to determine the `User` with the max age. However, note the use of generators and O(1) space complexity usage.

In [21]:
def reduce_max_age_user(acc, value):
    if value.age < acc.age:
        return acc
    return value    

In [22]:
functools.reduce(reduce_max_age_user, to_users())

User(user_id=7, age=87, first_name='Paul', favorite_color='red')

# 5. Composing Functions: One Final Tool to add to our toolbox

Often there is a need to chain to smaller functions together to create a new function. This is particularly useful for `pure` functions. 

For example we don't want to have to do this.

In [23]:
def squarer(a):
    return a * a

def doubler(a):
    return 2 * a

def incrementer(a):
    return a + 1

def forgetting_basics_of_programming():
    # This is terrible for so many reasons
    a = squarer(doubler(1))
    b = squarer(doubler(2))
    c = squarer(doubler(3))

    abc = [a,b,c]
    return abc

In [24]:
forgetting_basics_of_programming()

[4, 16, 36]

There's a lot of chatter and duplication that want to avoid in the example above. 

We could define a new func to do compose `squarer` and `doubler`. However, this is a very general need. Composing also often requires `N` number of functions. 

First, let's define `compose` to compose `N` functions using our closure technique combined with the `reduce` technique we've just added to our toolbox.

In [25]:
def compose(*funcs):
    """
    Functional composition
    
    [f, g, h] will be f(g(h(x)))
    """
    def compose_two(f, g):
        def c(x):
            return f(g(x))
        return c
    return functools.reduce(compose_two, funcs)

Let's test it out

In [26]:
fx = compose(incrementer, doubler, squarer)

print(list(map(fx, (1, 2, 7))))

[3, 9, 99]


Nice. Let's revisit the `forgetting_basics_of_programming` example. 

In [27]:
def getting_better():
    f = compose(squarer, doubler)
    a = f(1)
    b = f(2)
    c = f(3)
    return [a, b, c]

This is better, but still has duplication that we can improve on.

In [28]:
def to_better():
    f = compose(squarer, doubler)
    return [f(x) for x in range(1, 4)]

In [29]:
getting_better(), to_better()

([4, 16, 36], [4, 16, 36])

## Example 

These `helloworld`-esque examples are useful to demonstrate the idea, however, let's demonstrate the pattern in another real world context.

Let's say we have a CLI tool that has 3 different subparsers or separate commandline tools (e.g, alpha, beta, gamma). Each subparser or tool shares a common set of base commandline arguments, while `alpha` and `gamma` share a common subset of args. 

Let's use our composition model to construct an argparse.ArgumentParser.

The general structure will be to create functions that take our parser instance and return a parser. These aren't pure functions, however, we can still use the composition model to centralize functionality in our application.

In [30]:
def _to_log_opt(p):
    p.add_argument('--level', help="Logging Level", default="INFO")
    return p

def _to_opt_xy(p):
    p.add_argument('x', type=int, default=10, help="X Value")
    p.add_argument('y', type=int, default=20, help="Y Value")
    return p

def _to_opt_z(p):
    p.add_argument('z', type=int, default=30, help="Z Value")
    return p

def to_alpha_parser(name):
    p = argparse.ArgumentParser(description="Test Tool {}".format(name))
    f = compose(_to_opt_z, _to_opt_xy, _to_log_opt)
    return f(p)    

In [31]:
p = to_alpha_parser("Alpha")

In [32]:
p.parse_args("1 2 3 --level=DEBUG".split())

Namespace(level='DEBUG', x=1, y=2, z=3)

Instead of using OO patterns we're using function composition as core model for code reuse. 

For completeness, let's build out the the parser for the other three tools and remove some duplication along the way.

In [33]:
def _to_opt(achar, default, help):
    def func(p):
        p.add_argument(achar, type=int, default=default, help=help)
        return p
    return func

def _to_opt_xy(p):
    opts = [('X', 10, "X Value"), ('Y', 20, "Y Value")]
    f_opt = lambda x: _to_opt(*x)
    opt_funcs = map(f_opt, opts)
    f = compose(*opt_funcs)
    return f(p)

def to_parser(name, funcs):
    p = argparse.ArgumentParser(description="Test Tool {}".format(name))
    f = compose(*funcs)
    return f(p)    

def to_alpha_parser():
    funcs = (_to_opt_xy, _to_log_opt)
    return to_parser("Alpha", funcs)

def to_beta_parser():
    #let define this inline since it's not shared
    def f(p):
        p.add_argument('--min-radius', help="min radius", default=10)
        return p
    funcs = (f, _to_log_opt)
    return to_parser("Beta", funcs)

def to_gamma_parser():
    _to_opt_z = _to_opt('Z', 30, "Z Value")
    funcs = (_to_opt_z, _to_opt_xy, _to_log_opt)
    return to_parser("Gamma", funcs)

In [34]:
alpha_parser = to_alpha_parser()
# Note, argparse is kinda brutal with it's use of sys.exit. 
# this doesn't really play well with the notebook
alpha_parser.parse_args(["--help"])

usage: ipykernel_launcher.py [-h] [--level LEVEL] Y X

Test Tool Alpha

positional arguments:
  Y              Y Value
  X              X Value

optional arguments:
  -h, --help     show this help message and exit
  --level LEVEL  Logging Level


SystemExit: 0

/Users/mkocher/anaconda2/envs/core37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [35]:
alpha_parser.parse_args("1 2 --level=WARN".split())

Namespace(X=2, Y=1, level='WARN')

In [36]:
beta_parser = to_beta_parser()
beta_parser.parse_args("--min-radius 11".split())

Namespace(level='INFO', min_radius='11')

In [37]:
gamma_parser = to_gamma_parser()
gamma_parser.parse_args("1 2 3 --level=ERROR".split())

Namespace(X=2, Y=1, Z=3, level='ERROR')

Nice. We've constructed our parser for our 3 commandline tools using simple functions and a bit composition to share code and remove duplication along the way.

# Check Point

- We've gone over a few different models of defining a function in Python
- We've added closures to our toolbox of functional programming techniques
- We've added `functools.partial` from the Python standard library to provide us build up computation
- Constructed `compose` function that enables us to compose `N` different functions

We've got the basics functions and few new techniques in our toolbox, let's move to 2 remaining core funcs in the python standard lib to round out the fundaments of functional programming in Python.

# 6. Core Builtin Functions Map and Filter

Just to state the obvious. Every Python programming should be extremely familiar with the core builtin functions. 

https://docs.python.org/3/library/functions.html#built-in-functions

In the context of functional programming in Python, we're going to focus on `map` and `filter`. 

## Map 

https://docs.python.org/3/library/functions.html#map

This enables us to iterate over a list and apply a function and **return an Iterable**. The returning of an `iterable` enables composition. 

A few examples show below.

In [38]:
def f(x): 
    return x * x

rx = map(f, XS)
rx

In [39]:
isinstance(rx, Iterable)

True

In [40]:
list(map(f, range(1, 10)))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

Sometimes we'll have to sort out the impedence mismatch of the function signature to leverage `map`

In [41]:
import math
def to_distance(x, y, x0=0, y0=0):
    def f(a, a0):
        return math.pow(a - a0, 2)
    return math.sqrt(f(x, x0) + f(y, y0))

def example():
    points = [(3, 4), (5, 12)]
    f = lambda x: to_distance(*x)
    return map(f, points)

In [42]:
to_distance(3, 4)

5.0

In [43]:
list(example())

[5.0, 13.0]

Or to drag around context. 

For example, for a given set of points, determine the closest point to the origin. 

In [44]:
def example():
    points = ((5, 12), (6, 8), (3, 4))
    # let's return a tuple so we have the context of that data points
    # were used. 
    f = lambda x: (to_distance(*x), x)
    mx = map(f, points)
    # this is arguably a bit sloppy and abusing tuple sorting. 
    return functools.reduce(min, mx)

In [45]:
example()

(5.0, (3, 4))

## Filter

The next core util from the standard lib is `filter`. 

https://docs.python.org/3/library/functions.html#filter

This enables us to filter out specific data with a custom function. Similar to `map`, this returns an `Iterable`. 

Let's define a custom filter.

In [46]:
def custom_filter(x):
    return (x % 2) == 0 and x >= 2

In [47]:
fx = filter(custom_filter, range(0, 10))
isinstance(fx, Iterable)


True

In [48]:
list(fx)

[2, 4, 6, 8]

Let's try to make the original filter configurable with the min value 

In [49]:
def to_filter(min_value):
    def f(x):
        return (x % 2) == 0 and x >= min_value
    return f

In [50]:
f = to_filter(2)
list(filter(f, range(-2, 10)))

[2, 4, 6, 8]

Alternatively, we can write the function in a more "natural" form and create a function using partial application (as described in the previous section).

In [51]:
def custom_filter(min_value, x):
    return (x % 2) == 0 and x >= min_value

In [52]:
f = functools.partial(custom_filter, 2)

list(filter(f, XS))

[2, 4, 6, 8]

Defining Multiple Filters


In [53]:
def f1(x):
    return (x % 2) == 0

def example():
    f = functools.partial(custom_filter, 4)
    
    # Steps in the pipeline
    s1 = filter(f, range(1, 10))
    s2 = filter(f1, s1)
    return s1

print(example())
list(example())

[4, 6, 8]

Alternatively we can define a new custom compose filter function so that there's a single filter applied.

In [54]:
def compose_two_filters(f, g):
    def func(x):
        if g(x):
            return f(x)
        return False
    return func

def example():
    filter_min_4 = functools.partial(custom_filter, 4)
    f = compose_two_filters(f1, filter_min_4)
    return filter(f, XS)

list(example())

[4, 6, 8]

As an exercise, it's useful to define a compose filter function that takes `N` filter functions. 

In [55]:
def compose_filter(funcs):
    def compose_two_filters(f, g):
        def func(x):
            if g(x):
                return f(x)
            return False
        return func
    return functools.reduce(compose_two_filters, funcs)

### CheckPoint

- We've added the `map` and `filter` core stdlib builtin functions to our toolbox.
- With `map` sometimes it's necessary to sort out impedence mismatches and write small translation funcs.
- Leveraging the iterable nature of `map` and `filter` we can compose computation. 

Now that we've got the fundamentals down and we've added `map` and `filter` to our toolbox, we can now start building data processing pipelines!


# 7. Creating Pipelines

### Iteration 1:

Let's construct a pipeline that takes a stream (i.e., iterable) of `User` instances and filters the users by an attribute of the `User` and computes the user with largest age. Let's also make our pipeline configurable such that the user filter can be passed as input. 

This translates to this pipeline:

```
(iterable of users) -> (filter:by user metadata)
                    -> (reduce: to max user by age)
                    ->  User
```

In [56]:
def reduce_max_age_user(acc, value):
    if value.age < acc.age:
        return acc
    return value    

def filter_user_by_name(achar):
    def func(user):
        return user.first_name.startswith(achar)
    return func


def to_pipeline(users_it, user_filter):
    # Simple pipeline will only one step. 
    # Note, this doesn't store the entire user's list in memory.
    s1 = filter(user_filter, to_users())
    result = functools.reduce(reduce_max_age_user, s1)
    return result
    

filter_s_users = filter_user_by_name('S')    
to_pipeline(to_users(), filter_s_users)


User(user_id=3, age=66, first_name='Steve', favorite_color='orange')

Nice. We've leveraged several tools in our toolbox and have a pretty useful data pipeline to process the stream of `User`s. Let's expand on this in next iteration.

### Iteration 2:

Let's expand the first pipeline to return the Top `N` User sorted by age. 

There's several mechanisms to get the top `N` values. We'll use a `max heap` to do this by leveraging `heapq` in the Python standard lib. To get the `heap` sorting to work as expected, we'll have to add a few transformations to `tuples`. This is very similar to the sorting mechanism used in the closest points to the origin problem demonstrated in a previous section.

The new pipeline will approximately be translated to this pipeline:

```
(iterable of users) -> (filter:by user metadata)
                    -> (map: (age, user)) 
                    -> (reduce: to N users by age)
                    ->  list[(int, User)] 
                    -> map( (age, user) -> user) 
                    -> list[User]
```

Note that due to the use of the `heap` we've got to some extra `map` calls to transform the data to work with the heap. Similarly to 'untuple' the data using a `map` call to get the list of `User` instances. 

Also note the "reduce" step is not calling `functools.reduce` directly, but providing a similar interface in the pipeline. 

First let's implement the a mechanism to get the max N users. This is our "reduce". The "reduce" will take an iterable and return a list of top N `User`s.

In [57]:
import heapq as H

# https://docs.python.org/3.7/library/heapq.html

def compute_max_n(n):
    def f(it):
        h = []
        H.heapify(h)
        for item in it:
            if len(h) < n:
                H.heappush(h, item)
            else:
                # this will keep the heap to size `n`
                # Therefore keeping the space complexity to approximately O(1)
                _ = H.heappushpop(h, item)
        return H.nlargest(n, h)
    return f

In [58]:
def user_to_tuple(u):
    # for the heap
    return u.age, u

def from_tuple(x):
    return x[1]

def to_pipeline(users_it, user_filter, max_users):
    
    reducer = compute_max_n(max_users)
    
    # Note, the map and filter are Iterables
    p1 = filter(user_filter, users_it)
    p2 = map(user_to_tuple, p1)
    
    # this is effectively the reduce step,
    # however, it results a list
    p3 = reducer(p2)
    
    # transform back to list[User]
    p4 = map(from_tuple, p3)
    return p4
    

In [59]:
list(to_pipeline(to_users(), filter_s_users, 10))

[User(user_id=3, age=66, first_name='Steve', favorite_color='orange'),
 User(user_id=8, age=55, first_name='Stephen', favorite_color='blue'),
 User(user_id=5, age=4, first_name='Sam', favorite_color='blue'),
 User(user_id=9, age=2, first_name='Sean', favorite_color='blue')]

In [60]:
list(to_pipeline(to_users(), filter_user_by_name('R'), 3))

[User(user_id=4, age=41, first_name='Rebecca', favorite_color='blue'),
 User(user_id=6, age=32, first_name='Richard', favorite_color='blue'),
 User(user_id=1, age=13, first_name='Ralph', favorite_color='blue')]

Nice. Our pipeline appears to be working as expected and is quite configurable. Want to filter by all Users who's age is `>= 21` and name length is `< 10`? This only requires writing a custom function to do the user filtering. 

# 8. Conclusion and Summary

We've built up a toolbox of useful techinques using Functional Techinques using Python.

Recap:

- We've gone over a few different models of defining a function in Python
- We've added closures to our toolbox of functional programming techniques
- We've added `functools.partial` from the Python standard library to provide us build up computation
- Constructed `compose` function that enables us to compose `N` different functions
- We've added the `map` and `filter` core stdlib builtin functions to our toolbox.
- With `map` sometimes it's necessary to sort out impedence mismatches and write small translation funcs.
- Leveraging the iterable nature of `map` and `filter` we can compose computation. 
- Created extensible pipelines leveraging the `map`, `filter` and `reduce`

In Part 2, we'll go through an example of building a client to REST API using Functional style design. 

Hopefully you've picked up some tricks or solidified current your knowledge of Functional Techniques using Python.

Best to you and your Python-ing!

### Further Reading and Resources

- https://docs.python.org/3/library/functions.html 
- https://docs.python.org/3.7/howto/functional.html
- https://toolz.readthedocs.io/en/latest/index.html

In [61]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-15 03:20:20.292023
